In [1]:
with open("archives/funes.txt") as f: txt = f.read().splitlines()
while "" in txt: txt.remove("")

txt

['\ufeffFunes el memorioso',
 'Jorge Luis Borges\xa0',
 '\tLo recuerdo (yo no tengo derecho a pronunciar ese verbo sagrado, sólo un hombre en la tierra tuvo derecho y ese hombre ha muerto) con una oscura pasionaria en la mano, viéndola como nadie la ha visto, aunque la mirara desde el crepúsculo del día hasta el de la noche, toda una vida entera. Lo recuerdo, la cara taciturna y aindiada y singularmente remota, detrás del cigarrillo. Recuerdo (creo) sus manos afiladas de trenzado. Recuerdo cerca de esas manos un mate, con las armas de la Banda Oriental; recuerdo en la ventana de la casa una estera amarilla, con un vago paisaje lacustre. Recuerdo claramente su voz; la voz pausada, resentida y nasal del orillero antiguo, sin los silbidos italianos de ahora. Más de tres veces no lo vi; la última, en 1887... Me parece muy feliz el proyecto de que todos aquellos que lo trataron escriban sobre él; mi testimonio será acaso el más breve y sin duda el más pobre, pero no el menos imparcial del v

In [2]:
# %pip install -U sentence-transformers

In [3]:
import sentence_transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
embeddings = model.encode(txt)
embeddings

c:\Users\nicol\miniconda3\envs\ia\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\nicol\miniconda3\envs\ia\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


array([[ 1.1161916 , -0.5944312 , -0.16006903, ...,  0.04201887,
         0.1745641 , -0.13278355],
       [ 0.7258061 , -0.29302803,  0.15734741, ..., -0.5328277 ,
        -1.0715348 , -0.32097504],
       [-0.24173339, -0.39351282, -0.00498039, ..., -0.3861189 ,
        -0.3745569 , -0.16619876],
       ...,
       [-0.276651  ,  0.38812798,  0.12186418, ..., -0.56073517,
        -0.44604817,  0.1073512 ],
       [ 1.034237  ,  0.01015016, -0.1277492 , ..., -0.087633  ,
        -0.02246984,  0.08189754],
       [-0.6909963 , -0.35155368, -0.6079376 , ...,  0.7914548 ,
         0.2633531 , -0.21762164]], dtype=float32)

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Sentences we want sentence embeddings for
def get_sentence_embedding(sentences):
    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('hiiamsid/sentence_similarity_spanish_es')
    model = AutoModel.from_pretrained('hiiamsid/sentence_similarity_spanish_es')

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    return mean_pooling(model_output, encoded_input['attention_mask'])

org_embeddings = get_sentence_embedding(txt)
print(f"Sentence embeddings: \n{org_embeddings}")

Sentence embeddings: 
tensor([[ 1.1162, -0.5944, -0.1601,  ...,  0.0420,  0.1746, -0.1328],
        [ 0.7258, -0.2930,  0.1573,  ..., -0.5328, -1.0715, -0.3210],
        [-0.2417, -0.3935, -0.0050,  ..., -0.3861, -0.3746, -0.1662],
        ...,
        [-0.2767,  0.3881,  0.1219,  ..., -0.5607, -0.4460,  0.1074],
        [ 1.0342,  0.0102, -0.1277,  ..., -0.0876, -0.0225,  0.0819],
        [-0.6910, -0.3516, -0.6079,  ...,  0.7915,  0.2634, -0.2176]])


In [5]:
TextBuscar="muchacho del callejón"

In [6]:
new_embedding = get_sentence_embedding([TextBuscar])

# Calcular la similitud entre el texto nuevo y el texto original
similarities = torch.nn.functional.cosine_similarity(new_embedding, org_embeddings)

# Encuentra el párrafo más similar
index = torch.argmax(similarities)
most_similar = txt[index]

In [7]:
most_similar.strip()

"Me dijo que el muchacho del callejón era un tal Ireneo Funes, mentado por algunas rarezas como la de no darse con nadie y la de saber siempre la hora, como un reloj. Agregó que era hijo de una planchadora del pueblo, María Clementina Funes, y que algunos decían que su padre era un médico del saladero, un inglés O'Connor, y otros un domador o rastreador del departamento del Salto."